In [ ]:
!pip install xlsxwriter pydrive

In [7]:
#%% import
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import json
import re


#%% functions

def try_sitemap(base_url):
    """
    Próbuje pobrać linki z sitemap.xml
    """
    sitemap_urls = [
        f"{base_url}sitemap.xml",
        f"{base_url}sitemap_index.xml",
        f"{base_url}wp-sitemap.xml"
    ]

    for sitemap_url in sitemap_urls:
        try:
            r = requests.get(sitemap_url, timeout=10)
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'xml')
                links = [loc.text.strip() for loc in soup.find_all('loc')]
                if links:
                    print(f"✓ Znaleziono sitemap: {sitemap_url}")
                    return links
        except:
            continue

    return None


def get_links_from_page(page_url):
    """
    Pobiera linki z pojedynczej strony
    """
    try:
        r = requests.get(page_url, timeout=10)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'lxml')

        links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            # Normalizuj URL
            if href.startswith('/'):
                href = 'http://majcherek.waw.pl' + href
            elif not href.startswith('http'):
                href = 'http://majcherek.waw.pl/' + href

            if 'majcherek.waw.pl' in href:
                links.append(href)

        return links
    except Exception as e:
        print(f"Błąd dla {page_url}: {e}")
        return []


def find_pagination(soup, base_url):
    """
    Szuka linków do kolejnych stron
    """
    pages = []

    # Szukamy paginacji
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Typowe wzorce paginacji
        if any(pattern in href for pattern in ['/page/', '/strona/', '?page=', '?p=']):
            if href.startswith('/'):
                href = base_url.rstrip('/') + href
            pages.append(href)

    return pages


def get_all_article_links(base_url):
    """
    Główna funkcja - pobiera wszystkie linki
    """
    print("Krok 1: Próba pobrania sitemap...")
    sitemap_links = try_sitemap(base_url)

    if sitemap_links:
        print(f"Znaleziono {len(sitemap_links)} linków w sitemap")
        return sitemap_links, []

    print("Brak sitemap, scrapiuję strony...")

    # Scrapowanie stron
    all_links = set()
    pages_to_check = [base_url]
    checked_pages = set()

    print("\nKrok 2: Scrapowanie stron...")

    while pages_to_check and len(checked_pages) < 50:  # Max 50 stron
        page_url = pages_to_check.pop(0)

        if page_url in checked_pages:
            continue

        print(f"  Sprawdzam: {page_url[:60]}...")
        checked_pages.add(page_url)

        try:
            r = requests.get(page_url, timeout=10)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text, 'lxml')

            # Pobierz linki
            links = get_links_from_page(page_url)
            all_links.update(links)

            # Szukaj paginacji
            new_pages = find_pagination(soup, base_url)
            for p in new_pages:
                if p not in checked_pages:
                    pages_to_check.append(p)

            time.sleep(0.5)
        except Exception as e:
            print(f"    Błąd: {e}")

    return list(all_links), []


def filter_article_links(all_links):
    """
    Filtruje linki - zostawia tylko artykuły
    """
    article_links = []
    excluded = {'home': 0, 'static': 0, 'media': 0, 'short': 0, 'other': 0}

    for link in all_links:
        # Normalizuj - usuń fragmenty (#comments, #respond itp.)
        link = link.split('#')[0]  # Usuwa wszystko po #
        link = link.rstrip('/')

        # Pomiń puste
        if not link:
            continue

        # 1. Wykluczamy stronę główną
        if link == 'http://majcherek.waw.pl' or link.endswith('.pl/') or link.endswith('.pl'):
            excluded['home'] += 1
            continue

        # 2. Wykluczamy media i pliki
        if any(ext in link.lower() for ext in ['.jpg', '.jpeg', '.png', '.gif', '.pdf', '.zip', '.css', '.js', '.xml', '.ico']):
            excluded['media'] += 1
            continue

        # 3. Wykluczamy typowe strony statyczne i katalogi
        excluded_paths = [
            '/kontakt', '/about', '/o-blogu', '/o-autorze', '/archiwum',
            '/tag/', '/category/', '/kategoria/', '/autor/', '/author/',
            '/rss', '/feed', '/admin', '/wp-admin', '/wp-content', '/wp-includes',
            '/search', '/szukaj', '?s=', '?search=',
            '/page/', '/strona/', '?page=', '?p=',
        ]
        if any(x in link.lower() for x in excluded_paths):
            excluded['static'] += 1
            continue

        # 4. Wykluczamy zbyt krótkie URLe (prawdopodobnie nie artykuły)
        # Artykuły zwykle mają dłuższe URLe z tytułami
        path = link.replace('http://majcherek.waw.pl/', '').strip('/')
        if len(path) < 10:  # Zbyt krótki path
            excluded['short'] += 1
            continue

        # 5. Wykluczamy zewnętrzne linki (gdyby jakieś się dostały)
        if 'majcherek.waw.pl' not in link:
            excluded['other'] += 1
            continue

        # 6. AKCEPTUJEMY tylko jeśli wygląda jak artykuł
        # - Ma odpowiednią długość
        # - Nie zawiera parametrów GET (?, &)
        # - Ma sensowny path
        if '?' not in link and '&' not in link:
            # Sprawdź czy to nie jest tylko katalog
            if not link.endswith('/'):  # Katalogi często kończą się na /
                article_links.append(link)
            elif len(path.split('/')) >= 2:  # Jeśli ma głębszą strukturę
                article_links.append(link)

    # Usuń duplikaty (bo po usunięciu #comments mogą być duplikaty)
    article_links = list(set(article_links))

    print(f"\nStatystyki filtrowania:")
    print(f"  Strona główna: {excluded['home']}")
    print(f"  Strony statyczne/katalogi: {excluded['static']}")
    print(f"  Media/pliki: {excluded['media']}")
    print(f"  Zbyt krótkie URLe: {excluded['short']}")
    print(f"  Inne: {excluded['other']}")
    print(f"  ✅ ZAAKCEPTOWANO (bez duplikatów): {len(article_links)}")

    return article_links


#%% main execution

if __name__ == "__main__":
    base_url = "http://majcherek.waw.pl/"

    print("="*60)
    print("Pobieranie linków z majcherek.waw.pl")
    print("="*60)

    # Pobierz wszystkie linki
    all_links, errors = get_all_article_links(base_url)

    print(f"\nZnaleziono {len(all_links)} linków")

    # Filtruj
    article_links = filter_article_links(all_links)

    if not article_links:
        print("\n⚠️  Nie znaleziono artykułów!")
        print("Sprawdź ręcznie stronę:")
        print(f"  {base_url}")
        exit(1)

    # Pokaż przykłady
    if article_links:
        print("\nPrzykładowe linki (pierwsze 10):")
        for i, link in enumerate(sorted(article_links)[:10], 1):
            print(f"  {i}. {link}")

        if len(article_links) > 10:
            print(f"  ... i {len(article_links) - 10} więcej")

        # Opcja weryfikacji
        print("\n" + "="*60)
        response = input("Czy linki wyglądają dobrze? (t/n): ").strip().lower()
        if response == 'n' or response == 'nie':
            print("\n💡 Wskazówki:")
            print("  1. Sprawdź plik majcherek_linki.txt")
            print("  2. Usuń ręcznie niechciane linki")
            print("  3. Uruchom majcherek_scraper.py")
            # Zapisz mimo wszystko
            print("\n  Zapisuję mimo wszystko - możesz edytować plik...")
        print("="*60 + "\n")

    # Sortuj
    article_links.sort()

    # Zapisz do pliku tekstowego
    with open('majcherek_linki.txt', 'w', encoding='utf-8') as f:
        for link in article_links:
            f.write(link + '\n')

    # Zapisz do JSON
    output_data = {
        'source': base_url,
        'total_links': len(article_links),
        'links': article_links,
        'errors': errors
    }

    with open('majcherek_linki.json', 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    # Raport
    print("\n" + "="*60)
    print("RAPORT")
    print("="*60)
    print(f"Znaleziono artykułów: {len(article_links)}")
    print(f"\nZapisano do:")
    print(f"  - majcherek_linki.txt")
    print(f"  - majcherek_linki.json")
    print("="*60)

Pobieranie linków z majcherek.waw.pl
Krok 1: Próba pobrania sitemap...
Brak sitemap, scrapiuję strony...

Krok 2: Scrapowanie stron...
  Sprawdzam: http://majcherek.waw.pl/...
  Sprawdzam: http://majcherek.waw.pl/page/2/...
  Sprawdzam: http://majcherek.waw.pl/page/3/...
  Sprawdzam: http://majcherek.waw.pl/page/62/...
  Sprawdzam: http://majcherek.waw.pl/page/4/...
  Sprawdzam: http://majcherek.waw.pl/page/5/...
  Sprawdzam: http://majcherek.waw.pl/page/61/...
  Sprawdzam: http://majcherek.waw.pl/page/60/...
  Sprawdzam: http://majcherek.waw.pl/page/6/...
  Sprawdzam: http://majcherek.waw.pl/page/7/...
  Sprawdzam: http://majcherek.waw.pl/page/59/...
  Sprawdzam: http://majcherek.waw.pl/page/58/...
  Sprawdzam: http://majcherek.waw.pl/page/8/...
  Sprawdzam: http://majcherek.waw.pl/page/9/...
  Sprawdzam: http://majcherek.waw.pl/page/57/...
  Sprawdzam: http://majcherek.waw.pl/page/56/...
  Sprawdzam: http://majcherek.waw.pl/page/10/...
  Sprawdzam: http://majcherek.waw.pl/page/11/...

In [13]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    """
    try:
        date_string = ' '.join(date_string.strip().split())

        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        if 'T' in date_string:
            return date_string.split('T')[0]

        lookup_table = {
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        return format(changed_date.date())
    except Exception as e:
        print(f"Błąd konwersji daty '{date_string}': {e}")
        return "no date"


def dictionary_of_article(article_link):
    """
    Pobiera szczegóły artykułu z majcherek.waw.pl
    """
    try:
        r = requests.get(article_link, timeout=15)
        r.encoding = 'utf-8'
        html_text = r.text

        while '429 Too Many Requests' in html_text:
            time.sleep(5)
            r = requests.get(article_link, timeout=15)
            r.encoding = 'utf-8'
            html_text = r.text

        soup = BeautifulSoup(html_text, 'lxml')

        # Data publikacji - sprawdzamy różne miejsca
        try:
            date_element = None

            # Opcja 1: .entry-date (WordPress i inne)
            date_element = soup.find(class_='entry-date')

            # Opcja 2: <time> tag
            if not date_element:
                date_element = soup.find('time')

            # Opcja 3: Link z tekstem daty wewnątrz .entry-date
            # <li class="entry-date"><a href="...">11 listopada 2018</a></li>
            if not date_element:
                date_container = soup.find(['li', 'span', 'div'], class_='entry-date')
                if date_container:
                    date_link = date_container.find('a')
                    if date_link:
                        date_element = date_link
                    else:
                        date_element = date_container

            # Opcja 4: Link z tekstem daty (szukamy wszędzie)
            if not date_element:
                for link in soup.find_all('a', href=True):
                    link_text = link.get_text(strip=True)
                    # Sprawdź czy text wygląda jak data (ma miesiąc)
                    polish_months = ['stycznia', 'lutego', 'marca', 'kwietnia', 'maja', 'czerwca',
                                   'lipca', 'sierpnia', 'września', 'października', 'listopada', 'grudnia',
                                   'styczeń', 'luty', 'marzec', 'kwiecień', 'maj', 'czerwiec',
                                   'lipiec', 'sierpień', 'wrzesień', 'październik', 'listopad', 'grudzień']
                    if any(month in link_text.lower() for month in polish_months):
                        date_element = link
                        break

            # Opcja 5: <span> lub <div> z datą
            if not date_element:
                date_element = soup.find(['span', 'div'], class_=lambda x: x and 'date' in str(x).lower())

            # Opcja 6: Meta tagi
            if not date_element:
                meta_date = soup.find('meta', property='article:published_time')
                if meta_date:
                    date_element = type('obj', (object,), {
                        'get_text': lambda: meta_date.get('content', ''),
                        'get': lambda x: meta_date.get('content', '')
                    })()

            if date_element:
                date_text = date_element.get('datetime') or date_element.get('content') or date_element.get_text(strip=True)
                date_of_publication = date_change_format(date_text)
            else:
                date_of_publication = "no date"
        except Exception as e:
            print(f"Błąd parsowania daty dla {article_link}: {e}")
            date_of_publication = "no date"

        # Tytuł
        try:
            title_element = soup.find('h1')
            if not title_element:
                title_element = soup.find('title')

            title = title_element.get_text(strip=True) if title_element else "no title"
        except:
            title = "no title"

        # Autor - może być Majcherek lub inni
        try:
            author_element = soup.find(['span', 'div', 'a'], class_=lambda x: x and 'author' in str(x).lower())
            if not author_element:
                author_element = soup.find('a', rel='author')
            if not author_element:
                # Szukamy w meta
                meta_author = soup.find('meta', property='article:author')
                if meta_author:
                    author = meta_author.get('content', 'Janusz Majcherek')
                else:
                    author = "Janusz Majcherek"
            else:
                author = author_element.get_text(strip=True)
                author = re.sub(r'^(Autor|By|Opublikował):\s*', '', author, flags=re.IGNORECASE)
        except:
            author = "Janusz Majcherek"

        # Treść artykułu - uniwersalne
        try:
            # Próbujemy różnych selektorów
            article_body = soup.find('article')
            if not article_body:
                article_body = soup.find('div', class_=lambda x: x and ('content' in str(x).lower() or 'post' in str(x).lower() or 'entry' in str(x).lower()))
            if not article_body:
                article_body = soup.find('main')

            if article_body:
                text = article_body.get_text(strip=True).replace('\n', ' ').replace('\xa0', ' ')
            else:
                text = "no text"
        except:
            text = "no text"

        # Kategoria
        try:
            category_links = soup.find_all('a', rel='category')
            if not category_links:
                category_links = soup.find_all('a', class_=lambda x: x and 'category' in str(x).lower())

            if category_links:
                categories = [cat.get_text(strip=True) for cat in category_links]
                category = ' | '.join(categories)
            else:
                category = "no category"
        except:
            category = "no category"

        # Linki zewnętrzne
        try:
            if article_body:
                links = [a['href'] for a in article_body.find_all('a', href=True)]
                external_links = [link for link in links if not re.search(r'majcherek\.waw\.pl', link)]
                external_links = ' | '.join(external_links) if external_links else None
            else:
                external_links = None
        except (AttributeError, KeyError, IndexError):
            external_links = None

        # Zdjęcia
        try:
            images = []

            if article_body:
                content_images = [img['src'] for img in article_body.find_all('img', src=True) if img.get('src')]
                for img_src in content_images:
                    if img_src not in images:
                        images.append(img_src)

            has_images = len(images) > 0
            photos_links = ' | '.join(images) if images else None
        except (AttributeError, KeyError, IndexError):
            has_images = False
            photos_links = None

        # Filmy (iframe)
        try:
            if article_body:
                iframes = [iframe['src'] for iframe in article_body.find_all('iframe', src=True)]
                has_videos = len(iframes) > 0
            else:
                has_videos = False
        except:
            has_videos = False

        dictionary_of_article = {
            "Link": article_link,
            "Data publikacji": date_of_publication,
            "Tytuł artykułu": title.replace('\xa0', ' '),
            "Tekst artykułu": text,
            "Autor": author,
            "Kategoria": category,
            "Linki zewnętrzne": external_links,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(dictionary_of_article)

    except AttributeError as e:
        errors.append(article_link)
        print(f"Błąd dla {article_link}: {e}")
    except Exception as e:
        errors.append(article_link)
        print(f"Nieoczekiwany błąd dla {article_link}: {e}")


#%% main execution

if __name__ == "__main__":
    # Wczytaj linki z pliku
    try:
        with open('majcherek_linki.txt', 'r', encoding='utf-8') as f:
            article_links = [line.strip() for line in f if line.strip()]
        print(f"Wczytano {len(article_links)} linków z pliku")
    except FileNotFoundError:
        print("Nie znaleziono pliku majcherek_linki.txt")
        print("Użyj najpierw get_majcherek_links.py aby pobrać linki!")
        article_links = []

    if not article_links:
        print("Brak linków do przetworzenia!")
        exit(1)

    all_results = []
    errors = []

    print("\n" + "="*60)
    print("Rozpoczynam scraping artykułów z majcherek.waw.pl")
    print("="*60 + "\n")

    # Scraping z progress barem
    with ThreadPoolExecutor(max_workers=3) as executor:  # Mniej wątków dla mniejszej strony
        list(tqdm(executor.map(dictionary_of_article, article_links), total=len(article_links)))

    # Zapisywanie wyników
    timestamp = datetime.today().date()

    # JSON
    with open(f'majcherek_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)

    # Excel
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(f"majcherek_{timestamp}.xlsx",
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)

    # Raport
    print(f"\n{'='*60}")
    print(f"Scraping zakończony!")
    print(f"Przetworzono artykułów: {len(all_results)}")
    print(f"Błędów: {len(errors)}")
    if errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        if len(errors) > 10:
            print(f"  ... i {len(errors) - 10} więcej")
    print(f"\nPliki wyjściowe:")
    print(f"  - majcherek_{timestamp}.json")
    print(f"  - majcherek_{timestamp}.xlsx")
    print(f"{'='*60}\n")

Wczytano 369 linków z pliku

Rozpoczynam scraping artykułów z majcherek.waw.pl



100%|██████████| 369/369 [03:22<00:00,  1.82it/s]
/tmp/ipython-input-2616409096.py:287: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Posts', index=False)



Scraping zakończony!
Przetworzono artykułów: 369
Błędów: 0

Pliki wyjściowe:
  - majcherek_2026-01-12.json
  - majcherek_2026-01-12.xlsx



In [14]:
df.head()

,Link,Data publikacji,Tytuł artykułu,Tekst artykułu,Autor,Kategoria,Linki zewnętrzne,Zdjęcia/Grafika,Filmy,Linki do zdjęć
0,http://majcherek.waw.pl/10-postulatow-w-sprawi...,2014-10-07,10 postulatów w sprawie warszawskich teatrów –...,10 postulatów w sprawie warszawskich teatrówad...,admin,Bez kategorii,None,False,False,None
1,http://majcherek.waw.pl/10-lecie-kultury,2015-04-24,10-lecie Kultury – Blog Wojciecha Majcherka,"10-lecie Kulturyadmin,24 kwietnia 201510 lat t...",admin,Bez kategorii,None,False,False,None
2,http://majcherek.waw.pl/11-11-co-jest-grane,2018-11-11,11.11. – co jest grane? – Blog Wojciecha Majch...,"11.11. – co jest grane?admin,11 listopada 2018...",admin,Uncategorized,None,False,False,None
3,http://majcherek.waw.pl/13-lat-temu,2009-10-27,13 lat temu – Blog Wojciecha Majcherka,"13 lat temuadmin,27 października 2009W wydawni...",admin,Bez kategorii,None,False,False,None
4,http://majcherek.waw.pl/12-tylko-nie-placz-prosze,2022-06-07,12… tylko nie płacz proszę – Blog Wojciecha Ma...,"12… tylko nie płacz proszęadmin,7 czerwca 2022...",admin,Uncategorized,None,False,False,None
